# Uchwyty do obiektów

## Definicja 

> In computer programming, a handle is an abstract reference to a resource that is used when application software references blocks of memory or objects that are managed by another system like a database or an operating system.

### Czym uchwyty różnią się od wskaźników

> While a pointer contains the address of the item to which it refers, a handle is an abstraction of a reference which is managed externally; its opacity allows the referent to be relocated in memory by the system without invalidating the handle, which is impossible with pointers. The extra layer of indirection also increases the control that the managing system has over the operations performed on the referent.

### Przykład uchwytu

https://man7.org/linux/man-pages/man2/open.2.html

In [1]:
#include<stdio.h> 
#include<fcntl.h> 
#include <unistd.h>

int fd = open("/etc/passwd", O_RDWR);
close(fd);

In [2]:
struct Widget {};

In [3]:
Widget* findWidget(std::string) { return nullptr; }

In [4]:
Widget* getWidget(std::string name) {
    Widget *w;
    w = findWidget(name);
    return w;
}

In [5]:
void* getWidget(std::string name) {
    Widget *w;
    w = findWidget(name);
    return reinterpret_cast<void *>(w);
}

In [6]:
typedef void* HANDLE;

In [7]:
HANDLE getWidget(std::string name) {
    Widget *w;
    w = findWidget(name);
    return reinterpret_cast<HANDLE>(w);
}

https://docs.microsoft.com/en-gb/windows/win32/winprog/windows-data-types?redirectedfrom=MSDN

### Czas wrócić do cpp

- Czym są smart pointery?
- `std::shared_ptr`
  - Jak z nich korzystać?
  - (A)RC vs GC
  - Jak działa `std::shared_ptr`?
  - `std::make_shared` vs `new`
  - constructor aliasing
- `std::week_ptr`
  - Jak z nich korzystać?
  - Jakie problemy rozwiązują?
  - Czy na pewno `std::make_shared` jest lepsze od `new`?
- `std::unique_ptr`
  - Jak z nich korzystać?
  - `std::make_unique`
  - `std::shared_ptr` z `std::unique_ptr`
- Jak przekazywać smart pointery?
- Najczęściej popełniane błedy
- `std::enable_shared_from_this`
- `std::optional`
- Wartościowanie leniwe
  - funkcje konwertujące

### Czym są smart pointery?

> A smart pointer is a wrapper class over a pointer with an operator like * and -> overloaded. The objects of smart pointer class look like a pointer but can do many things that a normal pointer can’t like automatic destruction (yes, we don’t have to explicitly use delete), reference counting and more.

In [8]:
#include <memory>

### `std::shared_ptr`

> Use `std::shared_ptr` for shared-ownership resource management. ~ **Scott Meyers**

In [9]:
// ?std::shared_ptr

In [10]:
#include <iostream>

int* ptr = new int;
*ptr = 10;
std::cout << *ptr;

10

In [11]:
int* ptr = new int;
*ptr = 10;
std::cout << *ptr;
delete ptr; // <---------

10

In [12]:
{
    std::shared_ptr<int> sp(new int);
    *sp = 10;
    std::cout << *sp << std::endl; 
}

10


In [13]:
struct Foo {
    Foo() { std::cout << "Foo...\n"; }
    ~Foo() { std::cout << "~Foo...\n"; }
};

In [14]:
{
    std::shared_ptr<Foo> sp; 
}

In [15]:
{
    std::shared_ptr<Foo> sp(new Foo); 
}

Foo...
~Foo...


In [16]:
{
    auto* f = new Foo;
    std::shared_ptr<Foo> sp(f); 
}

Foo...
~Foo...


In [17]:
struct Foo2 {
    int i;
    Foo2(int i) : i(i) { std::cout << "Foo2[" << i << "]...\n"; }
    ~Foo2() { std::cout << "~Foo2[" << i << "]...\n"; }
};

In [18]:
{
    std::shared_ptr<Foo2> sp(new Foo2(1));
//     std::shared_ptr<Foo2> sp = new Foo2(1);

    std::shared_ptr<Foo2> sp2 = std::make_shared<Foo2>(2); 
}

Foo2[1]...
Foo2[2]...
~Foo2[2]...
~Foo2[1]...


In [19]:
struct Foo {
    int bar = 20;
    
    Foo() { std::cout << "Foo...\n"; }
    ~Foo() { std::cout << "~Foo...\n"; }
    
    void print() {std::cout << "Foo.print()\n"; }
};

In [20]:
std::shared_ptr<Foo> sp = std::make_shared<Foo>();
sp->print();
sp->bar = 30;
std::cout << sp->bar << std::endl;
(*sp).print();
std::cout << &(*sp) << std::endl;

Foo...
Foo.print()
30
Foo.print()
0x55a57d37ef10


#### .get()

In [21]:
std::cout << sp.get() << std::endl;

0x55a57d37ef10


#### .use_count()

In [22]:
sp.use_count()

1

#### .reset()

In [23]:
sp.reset()

~Foo...


In [24]:
sp.use_count()

0

In [25]:
std::cout << sp.get() << std::endl;

0


#### Kopiowanie wskaźników

In [26]:
{
    std::shared_ptr<Foo> sp;
    {
        std::shared_ptr<Foo> spInner = std::make_shared<Foo>();
        sp = spInner;
        
        std::cout << sp.use_count() << std::endl;
        
        std::cout << "End of inner scope \n";
    }
    
    std::cout << "End of scope \n";
}

Foo...
2
End of inner scope 
End of scope 
~Foo...


### (A)RC vs GC

![](./img/gc.png)

![](./img/rc.png)

### Ale w sumie to po co nam to?

In [27]:
#include <vector>

class A {
public:
    int a;
    
    A() {
        std::cout << "A...\n";
    }
    
    ~A() {
        std::cout << "~A...\n";    
    }
    
    virtual void print() {
        std::cout << "A\n";
    }
};

class B : public A {
public:
    int b;
    
    B() {
        std::cout << "B...\n";
    }
    
    ~B() {
        std::cout << "~B...\n";    
    }
    
    
    virtual void print() override {
        std::cout << "B\n";
    }
};

In [28]:
{ 
    std::vector<A> vec;

    vec.push_back(A());
    vec.push_back(B());

    for (auto& a : vec) {
        a.print();
    }
}

A...
~A...
A...
B...
~A...
~B...
~A...
A
A
~A...
~A...


In [29]:
{
    std::vector<A*> vec;

    vec.push_back(new A());
    vec.push_back(new B());

    for (auto* a : vec) {
        a->print();
    }
}

A...
A...
B...
A
B


In [30]:
{
    std::vector<std::shared_ptr<A>> vec;

    vec.push_back(std::make_shared<A>());
    vec.emplace_back(new A());
    vec.emplace_back(new B());

    for (auto a : vec) { // a może auto& <- ?
        a->print();
    }
}

A...
A...
A...
B...
A
A
B
~A...
~A...
~B...
~A...


### Jak to działa?

![](./img/sp-memory-foot-print.png)

In [31]:
class C : public B, public std::string {
    
};

{
    auto* c = new C();
    std::shared_ptr<C> sp(c);
    std::shared_ptr<A> sp2 = sp;
    std::shared_ptr<std::string> sp3 = sp;

    std::cout << "Raw: " << c << std::endl;
    
    std::cout << "sp: " << sp.get() << std::endl;
    std::cout << "sp2: " << sp2.get() << std::endl;
    std::cout << "sp3: " << sp3.get() << std::endl;
}

A...
B...
Raw: 0x55a57b627cf0
sp: 0x55a57b627cf0
sp2: 0x55a57b627cf0
sp3: 0x55a57b627d00
~B...
~A...


### `std::make_shared` vs `new`

![](./img/make_shared_vs_new.png)

### costructor aliasing

In [32]:
class Foo {
public:
    int f;
    
    Foo() {
        std::cout << "Foo...\n";
    }
    
    ~Foo() {
        std::cout << "~Foo...\n";    
    }
};

class Bar {
public:
    Foo foo;
        
    Bar() {
        std::cout << "Bar...\n";
    }
    
    ~Bar() {
        std::cout << "~Bar...\n";    
    }
};

In [33]:
{
    std::shared_ptr<Bar> barPtr = std::make_shared<Bar>();
    std::shared_ptr<Foo> fooPtr(barPtr, &barPtr->foo);
    
    std::cout << barPtr.use_count() << std::endl;
    std::cout << fooPtr.use_count() << std::endl;

    barPtr.reset();
    
    std::cout << barPtr.use_count() << std::endl;
    std::cout << fooPtr.use_count() << std::endl;

    fooPtr->f = 10;
}

Foo...
Bar...
2
2
0
1
~Bar...
~Foo...


### `std::weak_ptr`

In [34]:
// ?std::weak_ptr

> std::weak_ptr is a smart pointer that holds a non-owning ("weak") reference to an object that is managed by std::shared_ptr. It must be converted to std::shared_ptr in order to access the referenced object.

In [35]:
std::shared_ptr<Foo> ptr = std::make_shared<Foo>();

std::weak_ptr<Foo> weak = ptr;

if (std::shared_ptr<Foo> fromWeak = weak.lock()) {
    std::cout << "Foo wasn't destroyed\n";
} else {
    std::cout << "Foo was destroyed\n";
}

std::cout << weak.expired() << std::endl;

ptr.reset();

Foo...
Foo wasn't destroyed
0
~Foo...


In [36]:
std::shared_ptr<Foo> ptr = std::make_shared<Foo>();

std::weak_ptr<Foo> weak = ptr;

ptr.reset();

if (std::shared_ptr<Foo> fromWeak = weak.lock()) {
    std::cout << "Foo wasn't destroyed\n";
} else {
    std::cout << "Foo was destroyed\n";
}

std::cout << weak.expired() << std::endl;


Foo...
~Foo...
Foo was destroyed
1


### Jakie problemy rozwiązują?

In [37]:
class Base {
    
};

class F : public Base {
public:
    std::shared_ptr<Base> ptr;
    
    void setPtr(std::shared_ptr<Base> newPtr) {
        ptr = newPtr;
    }
    
    F() {
        std::cout << "Foo...\n";
    }
    
    ~F() {
        std::cout << "~Foo...\n";    
    }
};

class D : public Base {
public:
    std::shared_ptr<Base> ptr;
       
    void setPtr(std::shared_ptr<Base> newPtr) {
        ptr = newPtr;
    }
    
    D() {
        std::cout << "Bar...\n";
    }
    
    ~D() {
        std::cout << "~Bar...\n";    
    }
};

In [38]:
{
    std::shared_ptr<F> fPtr = std::make_shared<F>();
    std::shared_ptr<D> dPtr = std::make_shared<D>();
    
    fPtr->setPtr(dPtr);
    dPtr->setPtr(fPtr);
}

Foo...
Bar...


In [39]:
class FFix : public Base {
public:
    std::shared_ptr<Base> ptr;
    
    void setPtr(std::shared_ptr<Base> newPtr) {
        ptr = newPtr;
    }
    
    FFix() {
        std::cout << "Foo...\n";
    }
    
    ~FFix() {
        std::cout << "~Foo...\n";    
    }
};

class DFix : public Base {
public:
    std::weak_ptr<Base> ptr;
       
    void setPtr(std::shared_ptr<Base> newPtr) {
        ptr = newPtr;
    }
    
    DFix() {
        std::cout << "Bar...\n";
    }
    
    ~DFix() {
        std::cout << "~Bar...\n";    
    }
};

In [40]:
{
    std::shared_ptr<FFix> fPtr = std::make_shared<FFix>();
    std::shared_ptr<DFix> dPtr = std::make_shared<DFix>();
    
    fPtr->setPtr(dPtr);
    dPtr->setPtr(fPtr);
}

Foo...
Bar...
~Foo...
~Bar...


> Use `std::weak_ptr` for **shared_ptr**-like pointers that can dangle (and __know__ when they are dangling). ~ **Scott Meyers**

### Czy na pewno `std::make_shared` jest lepsze od `new`?

In [41]:
struct BigData {
    BigData() {
        std::cout << "BigData...\n";
    }
    
    ~BigData() {
        std::cout << "~BigData...\n";    
    }
};

In [42]:
{
    std::shared_ptr<BigData> sp = std::make_shared<BigData>();
    std::weak_ptr<BigData> weak = sp;
    
    sp.reset();
}

BigData...
~BigData...


In [43]:
{
    std::shared_ptr<BigData> sp(new BigData);
    std::weak_ptr<BigData> weak = sp;
    
    sp.reset();
}

BigData...
~BigData...


### `std::unique_ptr`

In [44]:
// ?std::unique_ptr

> Use `std::unique_ptr` for exclusive-ownership resource management. ~ **Scott Meyers**

In [45]:
{
    std::unique_ptr<A> up(new A);
}

A...
~A...


In [46]:
{
    std::unique_ptr<A> up(new A);
//     std::unique_ptr<A> up2 = up;
}

A...
~A...


In [47]:
{
    std::unique_ptr<A> up(new A);
    std::unique_ptr<A> up2 = std::move(up);
    
    std::cout << up.get() << std::endl;
    std::cout << up2.get() << std::endl;
}

A...
0
0x55a57e370630
~A...


### `std::make_unique`

In [48]:
{
    std::unique_ptr<int> up = std::make_unique<int>();
}

### `std::shared_ptr` z `std::unique_ptr`

In [49]:
{
    std::unique_ptr<std::string> uq = std::make_unique<std::string>("test");
    std::shared_ptr<std::string> sp = std::move(uq);
}

In [50]:
{
    std::vector<std::shared_ptr<A>> vec;

    vec.push_back(std::make_shared<A>());
    vec.emplace_back(new A());
    vec.emplace_back(new B());

    for (auto a : vec) { // a może auto& <- ?
        a->print();
    }
}

A...
A...
A...
B...
A
A
B
~A...
~A...
~B...
~A...


In [51]:
{
    std::vector<std::unique_ptr<A>> vec;

    vec.push_back(std::make_unique<A>());
    vec.emplace_back(new A());
    vec.emplace_back(new B());

    for (auto& a : vec) { // tu musi być auto&
        a->print();
    }
}

A...
A...
A...
B...
A
A
B
~A...
~A...
~A...


### Jak przekazywać smart pointery?

In [52]:
void t(const std::unique_ptr<std::string>& up) {
//     std::unique_ptr<std::string> temp = std::move(up);
}

In [53]:
void t2(std::unique_ptr<std::string>& up) {
    std::unique_ptr<std::string> temp = std::move(up);
}

In [54]:
void t3(std::unique_ptr<std::string> up) {
     std::unique_ptr<std::string> temp = std::move(up);
}

In [55]:
void t4(std::unique_ptr<std::string>&& up) {
     std::unique_ptr<std::string> temp = std::move(up);
}

In [56]:
class Moveable {
public:
    std::string id;
    
    Moveable(std::string id) : id(id) {}
    Moveable(Moveable&& other) {
        std::cout << "Move " << other.id << std::endl;
        id = std::move(other.id);
    }
}

In [57]:
void t(Moveable&& m) {}

In [58]:
Moveable m("m1");
t(std::move(m));
std::cout << m.id << std::endl;

m1


In [59]:
void t2(Moveable&& m) {
    Moveable inner = std::move(m);
}

In [60]:
Moveable m("m2");
t2(std::move(m));
std::cout << m.id << std::endl;

Move m2



### Najczęściej popełniane błędy

In [61]:
{
    std::shared_ptr<int> sp = std::make_shared<int>();
    std::weak_ptr<int> weak = sp;
    
    if (!weak.expired()) {
        auto fromWeak = weak.lock();
    }
}

In [62]:
auto* raw = new int;
std::shared_ptr<int> sp(raw);
std::shared_ptr<int> sp2(raw);

![](./img/cd.png)

In [63]:
{
    std::unique_ptr<int> ptr = std::make_unique<int>(10);
    int* v = ptr.release();
}

In [64]:
auto* raw = new int;
std::shared_ptr<int> sp(raw);
delete raw;

### `std::enable_shared_from_this`

In [65]:
struct Good: std::enable_shared_from_this<Good> {
    std::shared_ptr<Good> getptr() {
        return shared_from_this();
    }
};
 
struct Bad {
    std::shared_ptr<Bad> getptr() {
        return std::shared_ptr<Bad>(this);
    }
};

In [66]:
// Good: the two shared_ptr's share the same object
std::shared_ptr<Good> gp1 = std::make_shared<Good>();
std::shared_ptr<Good> gp2 = gp1->getptr();
std::cout << "gp2.use_count() = " << gp2.use_count() << '\n';

// Bad: shared_from_this is called without having std::shared_ptr owning the caller 
try {
    Good not_so_good;
    std::shared_ptr<Good> gp1 = not_so_good.getptr();
} catch(std::bad_weak_ptr& e) {
    // undefined behavior (until C++17) and std::bad_weak_ptr thrown (since C++17)
    std::cout << e.what() << '\n';    
}

// Bad, each shared_ptr thinks it's the only owner of the object
std::shared_ptr<Bad> bp1 = std::make_shared<Bad>();
std::shared_ptr<Bad> bp2 = bp1->getptr();
std::cout << "bp2.use_count() = " << bp2.use_count() << '\n';

gp2.use_count() = 2
bad_weak_ptr
bp2.use_count() = 1


### `std::optional`

In [67]:
// ?std::optional

In [68]:
#include <optional>

In [69]:
auto opt = std::optional<std::string>("Godzilla");

std::cout << opt.has_value() << std::endl;
if (opt) {
    std::cout << "Has value" << std::endl;
}

std::cout << opt.value() << std::endl;

std::cout << opt.value_or("Default value") << std::endl;

opt = std::nullopt;
if (!opt) {
    std::cout << "null" << std::endl;
} 

try {
    std::cout << opt.value() << std::endl;
} catch(std::bad_optional_access& e) {
    std::cout << e.what() << std::endl;
}

std::cout << opt.value_or("Default value") << std::endl;

1
Has value
Godzilla
Godzilla
null
bad optional access
Default value


### Leniwe wartościowanie

In [70]:
struct Point { 
    int x;
    int y; 
};

In [71]:
template<typename Left, typename Right>
struct AddOp {
    const Left& left;
    const Right& right;

    AddOp(const Left& left, const Right& right) : left(left), right(right) {}
};

In [72]:
struct Point { 
    int x;
    int y; 
    
    Point(int x, int y) : x(x), y(y) {}
    
    AddOp<Point, Point> operator+(const Point& right) {
        return AddOp<Point, Point>(*this, right);
    }
};

In [73]:
struct Point { 
    int x;
    int y; 
    
    Point(int x, int y) : x(x), y(y) {}
    
    template<typename Left, typename Right>
    Point(const AddOp<Left, Right>& op) {
        x = op.getX();
        y = op.getY();
    }

    template<typename Left, typename Right>
    Point& operator=(const AddOp<Left, Right>& op) {
        x = op.getX();
        y = op.getY();
        return *this;
    }
    
    AddOp<Point, Point> operator+(const Point& right) {
        return AddOp<Point, Point>(*this, right);
    }
    
    int getX() const { return x; }
    int getY() const { return y; }
};

In [74]:
template<typename Left, typename Right>
struct AddOp {
    const Left& left;
    const Right& right;

    AddOp(const Left& left, const Right& right) : left(left), right(right) {}
    
    int getX() const { return left.getX() + right.getX(); }
    int getY() const { return left.getY() + right.getY(); }
};

In [75]:
template<typename Left, typename Right> 
AddOp<AddOp<Left, Right>, Point> operator+(const AddOp<Left, Right>& left, const Point& p) {
    return AddOp<AddOp<Left, Right>, Point>(left, p);
} 

In [76]:
template<typename Left, typename Right> 
AddOp<Point, AddOp<Left, Right>> operator+(const Point& p, const AddOp<Left, Right>& rhs) {
    return AddOp<Point, AddOp<Left, Right>>(p, rhs);
}

In [77]:
Point p1{ 1, 2 };
Point p2{ 3, 4 };
Point p3{ 5, 6 };

// Point result = p1 + p2 + p3;


### Funkcje konwertujące

In [78]:
struct X {
    operator int() const { return 7; }
};

In [79]:
X x;

int intFromX = x;

In [80]:
intFromX

7

In [81]:
struct Y {
    explicit operator double() const { return 1.0; }
};

In [82]:
Y y;

// double doubleFromY = y; <- to nie zadziała 
// double doubleFromY = (double)y;
double doubleFromY = static_cast<double>(y);


In [83]:
doubleFromY

1.0000000